# Runner
Runner for the `ReqRespBunch` request list and callback implementation. Currently `CookieCallback` and `ResponseStoreCallback` are used as   cookie jar implementation and populating the response of each request.

In [ ]:
#| default_exp Runner

In [ ]:
#| export 
from FastRequest.ReqRespBunch import *
from FastRequest.RestReqFactory import *

In [ ]:
#| export 
#| hide
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

### Callback
- Basic callbacks for Runner. Callbacks can occur at any of these times *before_run*, *before_request*, *after_request*, *after_run*. Each `order` of the new derived callback should be greated that 0.
- Each class have the access to `epoch`, `resp` the response returned the respinse, `reqs_bunch` the req list, `resps_bunch` response list, `kwargs_bunch` kwargs dictionary.
- it also the access to both `set_kwargs` and `set_resp`

In [ ]:
#| export 
#| hide
import re
class Callback():
    _order=0
    
    def set_order(
        self,
        order:int=1 # new value of order defults to 1
    ):
        """
            set order checks if `order` > 1, sets prioriy of the callback
            use `self._order`, for setting up the running of callback 
        """
        if order  <  1:
            raise Exception("Order is lt 1")
            
        self._order = order
        
    def set_runner(self, run): self.run=run
        
    def __getattr__(self, k): return getattr(self.run, k)
    
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return name
    
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

### Runner
Init the runners with list of callbacks.

In [ ]:
#| export 
from tqdm.notebook import tqdm
from time import time 


class Runner():
    def __init__(self, cb_funcs=None):
        base_cb = [CookieCallback, ResponseStoreCallback]
        if not cb_funcs:
            cb_funcs = base_cb
        else:
            cb_funcs = base_cb + cb_funcs
        cbs = []
        for cfs in listify(cb_funcs):

            cb = cfs()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.cbs = cbs
    
    @property
    def reqs_bunch(self):  return self.bunchs.reqs

    @property
    def resps_bunch(self):  return self.bunchs.resps

    @property
    def kwargs_bunch(self):  return self.bunchs.kwargs

    def set_kwargs(self, idx, val):
        self.bunchs.set_kwargs(idx, val)

    def set_resp(self, idx, val):
        self.bunchs.set_resp(idx, val)
    
    def send_one_req(
        self,
        req:RestReq,       # invoke rest request 
        debug:bool=False,  # debug  
        **kwargs
    ):
        "invoke request of `req` with kwargs containing cookies and redirect"
        if debug:
                print(f"{'====='*7} epoch={self.epoch+1} {'====='*7}")
                print(f"request:\n{'-'*len('request:')}\n{req()}")
                print(f"kwargs:\n{'-'*len('kwargs:')}\n{kwargs}")
                
        
        time_ = time()
        self.resp = req()(**kwargs)
        
        
        if debug:
            print(f"execution time: {(time()-time_)*10**3:.2f}ms" )
            print(f"response:\n{'-'*len('response:')}\n{self.resp}")
        
    def run(
        self, 
        bunchs:ReqRespBunch,  # itrate over all the request object and send response
        debug:bool=False     # debug each itration
    ):
        "run method to  iter over all the ReqRespBunch's reqs and populate the resp as well as kwargs"
        self.bunchs, self.epoch  = bunchs,0

        for cb in self.cbs: cb.set_runner(self)

        self('before_run')
        
        for i  in  tqdm(range(len(self.bunchs))):
            
            self.req = self.reqs_bunch[i]
            self.epoch = i
            
            self('before_request') 
            
            kwargs = self.kwargs_bunch[i] 
            
            self.reqs_bunch[i].set_req_kwargs(kwargs) 
            
            
            self.send_one_req(self.req, debug, **kwargs)
            
                
            self('after_request') 
        
        self('after_run')

    def __call__(self, cb_name):
        res = False
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) or res
        return res

Bellow are two callbacks we implemented with its functionality <br>
1. `CookieCallback` is used for chaining the cookies from a rasponse to setnext request's kwargs<br>
1. `ResponseStoreCallback` for storing each request's response

In [ ]:
#| export 
#| hide
class CookieCallback(Callback):
    def before_request(self):
        
        if self.epoch != 0:
            self.set_kwargs(self.epoch, ('cookies', self.resps_bunch[self.epoch - 1].cookies))
        
    def after_request(self):
        ...
class ResponseStoreCallback(Callback):
    def after_request(self):
        self.set_resp(self.epoch, self.resp)

In [ ]:
# Create the RestReqFactory instance
req1 = RestReqFactory(
    method="GET",
    url_provider= lambda: f"{get_env('url1')}/get",
    params_provider= lambda : {
        "foo1": f"{get_env('foo1')}",
        "foo2": f"{get_env('foo2')}",
    }
)
req2 = RestReqFactory(
    method="GET",
    url_provider= lambda : f"{get_env('url2')}/api/users/2",
)
reqs = ReqRespBunch([req1, req2])
len(reqs)

2

In [ ]:
set_env('foo1', 'foo1')
set_env('foo2', 'foo2')
set_env('url1', 'https://postman-echo.com')
set_env('url2', 'https://reqres.in')

In [ ]:
runner = Runner()
runner.run(reqs, debug=True)

  0%|          | 0/2 [00:00<?, ?it/s]

=================================== epoch=1 ===================================
request:
--------
{
    "method": "GET",
    "url": "https://postman-echo.com/get",
    "headers": "None",
    "params": {
        "foo1": "foo1",
        "foo2": "foo2"
    },
    "body": "None",
    "kwargs": "None"
}
kwargs:
-------
{}
execution time: 875.37ms
response:
---------
{
    "url": "https://postman-echo.com/get?foo1=foo1&foo2=foo2",
    "status": 200,
    "headers": {
        "Date": "Mon, 10 Jun 2024 14:32:22 GMT",
        "Content-Type": "application/json; charset=utf-8",
        "Content-Length": "512",
        "Connection": "close",
        "Server": "nginx/1.25.3",
        "ETag": "W/\"200-u8K/5We2ycAZZmfyGOJShXcIc88\"",
        "set-cookie": "sails.sid=s%3AbccKPy3Ni4wAGdPwtqitIniFOpF-JHp6.eRZq87hnH6nr%2FgaN%2BR6ug%2BX5sx6NpLtOEQj%2B%2BiZamIE; Path=/; HttpOnly"
    },
    "cookies": {
        "sails.sid": "s%3AbccKPy3Ni4wAGdPwtqitIniFOpF-JHp6.eRZq87hnH6nr%2FgaN%2BR6ug%2BX5sx6NpLtOEQj%2B%2

In [ ]:
#| hide
import nbdev

In [ ]:
#| hide
nbdev.export.nb_export('Runner.ipynb','../FastRequest/')